In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


final_df_list = []  # create an empty list to hold the final data frames

for league_id in range(50200, 50400):
    res = requests.get(f"https://draft.premierleague.com/api/league/{league_id}/details")
    data = res.json()

    if "matches" not in data:
        continue

    league_entries = data["league_entries"]

    if len(league_entries) != 16:
        continue

    columns = []
    for items in league_entries:
        columns.append(items["id"])

    row = [x + 1 for x in range(38)]
    df = pd.DataFrame(data, columns=columns, index=row)

    entry_names = {}
    for items in league_entries:
        entry_id = items["id"]
        entry_name = items["entry_name"]
        entry_names[entry_id] = entry_name

    new_columns = [entry_names[x] for x in columns]
    new_df = pd.DataFrame(data, columns=new_columns, index=row)

    for match in data["matches"]:
        event = match["event"]
        league_entry_1 = match["league_entry_1"]
        league_entry_1_points = match["league_entry_1_points"]
        league_entry_2 = match["league_entry_2"]
        league_entry_2_points = match["league_entry_2_points"]

        new_df.at[event, entry_names[league_entry_1]] = league_entry_1_points
        new_df.at[event, entry_names[league_entry_2]] = league_entry_2_points

    points_df = new_df.copy()
    points_df = new_df.astype(float)

    for match in data["matches"]:
        event = match["event"]
        if match["finished"] == False:
            league_entry_1 = entry_names[match["league_entry_1"]]
            league_entry_2 = entry_names[match["league_entry_2"]]
            points_df.at[event, league_entry_1] = np.nan
            points_df.at[event, league_entry_2] = np.nan
            continue
        league_entry_1 = entry_names[match["league_entry_1"]]
        league_entry_1_points = match["league_entry_1_points"]
        league_entry_2 = entry_names[match["league_entry_2"]]
        league_entry_2_points = match["league_entry_2_points"]

        if league_entry_1_points > league_entry_2_points:
            points_df.at[event, league_entry_1] = 3
            points_df.at[event, league_entry_2] = 0
        elif league_entry_1_points < league_entry_2_points:
            points_df.at[event, league_entry_1] = 0
            points_df.at[event, league_entry_2] = 3
        else:
            points_df.at[event, league_entry_1] = 1
            points_df.at[event, league_entry_2] = 1

    points_df_cumsum = points_df.cumsum()

    new_df = new_df.astype(float)
    new_df_cumsum = new_df.cumsum()

    cumulative_avg = new_df_cumsum.mean(axis=1)
    new_df_centered = new_df_cumsum.subtract(cumulative_avg, axis=0)

    standings = data['standings']
    team_diffs = {}
    for team in standings:
        team_name = entry_names[team['league_entry']]
points_for = team['points_for']
points_against = team['points_against']
diff = points_for - points_against
team_diffs[team_name] = diff

difference_points_df = pd.DataFrame.from_dict(team_diffs, orient='index', columns=['Point Difference'])
difference_points_df = difference_points_df.sort_values(by='Point Difference', ascending=False)
difference_points_df = difference_points_df.reset_index()
difference_points_df = difference_points_df.rename(columns={"index": "Team", "Point Difference": "Point Difference"})

# Sum up the total points for each team
leaderboard = points_df.sum().sort_values(ascending=False)

# Create a data frame from the leaderboard series
leaderboard_df = pd.DataFrame(leaderboard).reset_index()

# Rename the columns of the data frame
leaderboard_df.columns = ['Team', 'Points']

# Sum up the total points for each team
total_points = new_df.sum().sort_values(ascending=False)

# Create a data frame from the leaderboard series
total_points_df = pd.DataFrame(total_points).reset_index()

# Rename the columns of the data frame
total_points_df.columns = ['Team', 'Total Points']

# Create a data frame from the leaderboard series
trades_df2 = pd.DataFrame(total_points).reset_index()

# Rename the columns of the data frame
trades_df2.columns = ['Team', 'Total Trades']

# Merge total_points_df with leaderboard_df on 'Team' column
merged_df = total_points_df.merge(leaderboard_df, on='Team')

# Merge trades_df2 with the above merged dataframe on 'Team' column
merged_df = merged_df.merge(trades_df2, on='Team')
merged_df = merged_df.merge(difference_points_df, on='Team')

final_df_list.append(merged_df)

final_df = pd.concat(final_df_list, ignore_index=True)
final_df = final_df.sort_values('Points', ascending=False)
final_df['Rank'] = np.arange(1, len(final_df)+1)

points = final_df.pop('Points')

final_df.insert(final_df.columns.get_loc('Total Points'), 'Points', points)

final_df.to_csv('MYLEAGUES.csv', index=False)



In [16]:
final_df 


,Team,Points,Total Points,Total Trades,Point Difference,Rank
0,Marcs Brig 11,14.0,557.0,557.0,-266,1
